# Beautifulsoup & Requests 
 - Crawling을 위해서는 기본적으로 웹에서 데이터를 요청하고 요청한 데이터를 받는다 라는 것을 이해하고 있어야한다.
 - 데이터를 달라고 요청하는 것이 우선, 그 다음이 요청한 데이터를 받아서 우리가 하고 싶은 일들을 하는 것이다. 
 - 우선 데이터를 요청하는 라이브러리부터 학습하겠다.

## 1. Requests

In [1]:
import requests

 - 일반적인 웹 사이트의 header의 정보를 입력하지 않아도 내용을 가지고 올 수 있으나, tistory의 경우 브라우저에 따라 데이터를 접근할 수 있고 없고를 구분하는 것으로 볼 수 있다.
 - Response 403 : Forbidden
  - 접근 제한.
  - 이전의 Crawling Post에 Status Code를 업로드 해놓았음.
  - <a href='http://jtoday.tistory.com/84'> 이전 Post </a>

In [8]:
url = 'http://jtoday.tistory.com/80'
res = requests.get(url)
res

<Response [403]>

In [9]:
res = requests.get(url, headers = header)
res

<Response [200]>

In [13]:
res.headers # 이와 같이 res. 이후 Tap을 사용하게 되면 response 객체 내 어떠한 내용을 담고 있는지 확인 할 수 있다.

{'X-UA-Compatible': 'IE=Edge', 'Set-Cookie': 'visited=1505130443; path=/; domain=jtoday.tistory.com', 'Server': 'Apache', 'Access-Control-Allow-Origin': 'http://jtoday.tistory.com', 'Connection': 'Keep-Alive', 'Content-Length': '17107', 'Keep-Alive': 'timeout=5, max=92', 'Content-Encoding': 'gzip', 'Content-Type': 'text/html; charset=utf-8', 'P3P': "CP='ALL DSP COR MON LAW OUR LEG DEL'", 'Date': 'Mon, 11 Sep 2017 11:47:23 GMT', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Vary': 'Accept-Encoding', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Pragma': 'no-cache'}

In [14]:
res.url

'http://jtoday.tistory.com/80'

In [15]:
res.encoding # Encoding 정보 

'utf-8'

In [16]:
res.status_code # 이 정보를 통해 Automated Crawling 을 만들게 되면 Request 성공 여부를 구분하여 만들면 된다.

200

In [18]:
res.text[:100] # .text를 통해 html 태그를 String 형태로 확인 할 수 있다.

'<!doctype html>\n<html lang="ko">\n\n\n<head>\n<link rel="stylesheet" type="text/css" href="//t1.daumcdn.'

## 웹 동작, GET과 POST

 - GET : 서버의 자원을 요청할 때 즉, 영어 어원 그대로 무엇인가를 받을 때 사용한다. 
 - POST : 서버에 자원을 추가할 때 이 또한 Posting 한다와 같은 의미라고 생각하면 된다.
 - 정보를 획득할 때나 업로드 할 경우 둘다 사용자는 데이터를 제공하고 그것을 통해 해당 정보를 업로드하거나 획득하게 된다.
 - GET 
  - Idx = 3 인 정보를 테이블에서 가지고 와 주세요.
 - POST
  - Bookid = 3, title = '월스트리트저널 인포그래픽 가이드', author = '도나 M.윙' 의 정보를 입력해주세요. 
 - 주거나 또는 받거나의 행동이지만 Crawling 은 동일한 방법으로 수행 할 수 있다.

### GET

In [20]:
get_data = {'idx':3}
url = 'http://tistory.com'
res = requests.get(url,params=get_data)

### POST

In [ ]:
post_data = {'bookid':4, 'title':'월스트리트저널 인포그래픽 가이드', 'author'='도나 M.윙'}
res = requests.post(url, data=post_data) # 와 같은 방법으로 수행하면 된다.

## 2. BeautifulSoup 
 - 위의 Requests 를 통해 받은 정보를 사용자가 읽기 쉽게 원하는 데이터를 뽑기 위해 사용하는 HTML Parser 다.

In [27]:
from bs4 import BeautifulSoup

In [23]:
url = 'https://www.clien.net/service/group/board_all'
get_data = {'od':'T33'}
res = requests.get(url, params=get_data)
res

<Response [200]>

 - 이렇게 본다면 알아 보기도 힘들고 내가 원하는 정보를 볼 수도 없다.
 - 속성부터 CSS, JS까지 모든 정보가 출력이 된다.

In [24]:
res.text[:200]

'\n\n\n<!DOCTYPE HTML>\n<html lang="ko">\n<head>\n\t<title>클리앙 : 커뮤니티 전체</title>\n\t<meta name="description" content="클리앙의 모든 커뮤니티 게시판 모음 입니다." />\n\t<meta name="robots" content="noindex">\n\t<meta charset="utf-8">'

In [30]:
soup = BeautifulSoup(res.text, 'html.parser')
type(soup)

bs4.BeautifulSoup

In [38]:
type(soup.get_text()) # String 값으로 변환하여 가지고 온다.

str

In [31]:
soup.title

<title>클리앙 : 커뮤니티 전체</title>

### 아래의 목록에서 제목만 추출해보고 싶다.
 - 우선 HTML을 꾸며주는 CSS의 특성을 알아봐야한다.
 - 특정 다양한 A라는 태그 중 몇몇개의 특정 패턴이 있는 태그만 새로운 꾸밈을 주고 싶을 때 사용하는 것이 class와 id 라는 속성이다.
  - class : 동일한 이름의 속성을 여러곳에 사용할 수 있다.
  - id : 단일의 태그 속성이다. 즉, 여러 태그 중 1곳에만 사용할 수 있다.
 - HTML은 계층형 구조를 가지고 있다. 부모가 있고 그 안에 자식들이 분포하고 있는 구조라 부모로 부터 자식으로 까지 순차적으로도 찾아 갈 수도 있다.
![board1](./img/1.png)

### find, find_all
 - soup를 통해 parser한 내용 중 찾고 싶은 것이 있다면?
 - find : 찾으려는 태그 중 첫번쨰 1개만 찾는다.
 - find_all : 찾으려는 태그와 속성을 가진 모든 태그를 찾는다.

In [34]:
soup.find('a') # 1개의 a 태그를 찾는다.

<a href="/service/" title="클리앙홈 바로가기"><i class="icon-clien_logo"></i><span class="blind">CLIEN</span></a>

In [36]:
soup.find_all('a')[:5] # a 태그를 전부 찾고 Slice[5개만]

[<a href="/service/" title="클리앙홈 바로가기"><i class="icon-clien_logo"></i><span class="blind">CLIEN</span></a>,
 <a class="skip-to" href="#div_content">본문 바로가기</a>,
 <a class="skip-to" href="#div_sidemenu">메뉴 바로가기</a>,
 <a class="skip-to text-large" href="javascript:;">폰트크기변경</a>,
 <a class="skip-to button-dark-pc" href="javascript:;">색상반전보기</a>]

 - A 태그 중 class 값을 list-subject를 가진 태그를 찾고 싶다면?
 - find('태그',class_='class-Name') 를 수행

In [40]:
soup.find('a', class_='list-subject')

<a class="list-subject" data-role="cut-string" href="/service/board/rule/10707403?po=0&amp;od=T33&amp;sk=&amp;sv=&amp;category=&amp;groupCd=board_all&amp;articlePeriod=default" style="display:block;">
																사이트 이용규칙 (종합)
							</a>

In [42]:
soup.find_all('a', class_='list-subject')[:2]

[<a class="list-subject" data-role="cut-string" href="/service/board/rule/10707403?po=0&amp;od=T33&amp;sk=&amp;sv=&amp;category=&amp;groupCd=board_all&amp;articlePeriod=default" style="display:block;">
 																사이트 이용규칙 (종합)
 							</a>,
 <a class="list-subject" data-role="cut-string" href="/service/board/park/11180253?po=0&amp;od=T33&amp;sk=&amp;sv=&amp;category=&amp;groupCd=board_all&amp;articlePeriod=default" style="display:block;">
 <span class="subject-shortname">모공</span>
 															[릴레이 33일차]장충기문자
 				</a>]

### 해당 태그의 제목 또는 속성만 가지고 오기 

In [64]:
soup.find_all('a', class_='list-subject')[0].text

'\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t사이트 이용규칙 (종합)\n\t\t\t\t\t\t\t'

In [65]:
soup.find_all('a', class_='list-subject')[0].attrs['href']

'/service/board/rule/10707403?po=0&od=T33&sk=&sv=&category=&groupCd=board_all&articlePeriod=default'

In [63]:
for lists in soup.find_all('a', class_='list-subject'):
    print(lists.text.replace('\t','').replace('\n','').replace('모공','') + " \n 주소 www.client.com" + lists.attrs['href'])

사이트 이용규칙 (종합) 
 주소 www.client.com/service/board/rule/10707403?po=0&od=T33&sk=&sv=&category=&groupCd=board_all&articlePeriod=default
[릴레이 33일차]장충기문자 
 주소 www.client.com/service/board/park/11180253?po=0&od=T33&sk=&sv=&category=&groupCd=board_all&articlePeriod=default
망가진 언론의 피해자.jpg 
 주소 www.client.com/service/board/park/11181973?po=0&od=T33&sk=&sv=&category=&groupCd=board_all&articlePeriod=default
[펌] 한경오/정의당 : 우리 같은 진보 세력이잖아ㅠㅠ 도대체 왜 욕하는거야? 
 주소 www.client.com/service/board/park/11180188?po=0&od=T33&sk=&sv=&category=&groupCd=board_all&articlePeriod=default
박정희 그놈 일본한테 뇌물 먹었어요. 
 주소 www.client.com/service/board/park/11180443?po=0&od=T33&sk=&sv=&category=&groupCd=board_all&articlePeriod=default
조희연 "특수학교, 양보할 수 있는 사안 아니다" 
 주소 www.client.com/service/board/park/11179899?po=0&od=T33&sk=&sv=&category=&groupCd=board_all&articlePeriod=default
장애인주차장 신고했더니 건물측이 신고자색출하겠다네요 
 주소 www.client.com/service/board/park/11182268?po=0&od=T33&sk=&sv=&category=&groupCd=board_all&articlePeriod=default
문재인 대통